In [1]:
import configparser
import dataclasses
import os
import re
import shutil
import statistics
from collections import Counter
from itertools import chain
from pathlib import Path
from time import perf_counter

import pandas as pd
from Bio import SeqIO, pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
from IPython.display import display
import logging

import utils

In [24]:
# fns to check config files
    

def checking_dirs(dirname: str, log: logging, log_msg: bool=True, create_dir: bool=False):
    dirname = str(Path(dirname).resolve())
    res = utils.dir_exists_and_create(dirname, create_dir=create_dir)
    if res == 4 or res == 3:
        log.info(f"[+] INFO: {dirname} already exists") if log_msg else None
        return(dirname)
    elif res == 2:
        log.info(f"[X] WARNING: {dirname} doesn't exists, check the config file") if log_msg else None
        log.info(f"[X] ABORT") if log_msg else None
        os.abort()
    elif res == 1:
        log.info(f"[+] INFO: {dirname} created") if log_msg else None
        return(dirname)

def check_files(file_path: str, log: logging):
    file_path = str(Path(file_path).resolve())
    if utils.file_exists(file_path) == 0:
        log.info(f"[X] WARNING: {file_path} doesn't exists, check the config file")
        log.info(f"[X] ABORT")
        os.abort()
    else:
        log.info(f"[+] INFO: {file_path} exists")
        return(file_path)

def check_tsv_file(filepath: str, log: logging) -> bool:
    """
    Returns True if the tsv file had name and h3_nt columns and no missing values in it
    """
    df = pd.read_csv(filepath, sep="\t")
    if [i for i, j in zip(sorted(df.columns.tolist(), reverse=True), sorted(['name', 'h3_nt'], reverse=True)) if i == j] == ['name', 'h3_nt']:
        log.info(f"[+] INFO: {filepath} contains column names 'name' and 'h3_nt'")
        return df[['name', 'h3_nt']].dropna()
    else:
        log.info(f"[+] WARNING: {filepath} should contain column names 'name' and 'h3_nt'")
        log.info(f"[X] ABORT")
        os.abort()

def create_results_excel_file_path(res_fp:str, res_fn: str):
    return str(Path(os.path.join(res_fp, res_fn)).resolve())

def get_patterns(config: configparser.ConfigParser):
    pat_vh,pat_vl=config['Pattern']['pat_vh'], config['Pattern']['pat_vl']
    return pat_vh, pat_vl

def get_patterns_to_rm(config: configparser.ConfigParser):
    patrm_vh, patrm_vl = config['Pattern']['vh_pat_to_rm'], config['Pattern']['vl_pat_to_rm']
    return patrm_vh, patrm_vl

def checking_ab1_files(vh_abi_dict: dict, vl_abi_dict: dict) -> str:
    """
    Return sample ids
    """
    vh_keys = [i for i in [*vh_abi_dict.keys()]]
    vl_keys = [i for i in [*vl_abi_dict.keys()]]
    if len(vh_abi_dict) == len(vl_abi_dict):
        log.info(f"[+] INFO: Same number of samples (n={len(vh_abi_dict)}) in vh_abi_dict and vl_abi_dict")
        if vh_keys == vl_keys:
            log.info(f"[+] INFO: The sample names in vh_abi_dict abd vl_abi_dict are same")
            return vl_keys
        else:
            log.info(f"[+] WARNING: Some sample names in vh_abi_dict abd vl_abi_dict are different; check the patterns given in `vh_pat_to_rm` or `vl_pat_to_rm`")
            log.info(f"[X] ABORT")
            os.abort()
        
def get_abi_file_path(key:str, vh_abi_dict: dict, vl_abi_dict: dict) -> str:
    vh = vh_abi_dict.get(key)
    vl = vl_abi_dict.get(key)
    return vh, vl

def _abi_trim(seq_record):
    start = False   # flag for starting position of trimmed sequence
    segment = 20    # minimum sequence length
    trim_start = 0  # init start index
    cutoff = 0.01   # default cutoff value for calculating base score
    # calculate base score
    score_list = [cutoff - (10 ** (qual / -25.0)) for qual in seq_record.letter_annotations["phred_quality"]]
    # calculate cummulative score
    # if cummulative value < 0, set it to 0
    # first value is set to 0, because of the assumption that
    # the first base will always be trimmed out
    cummul_score = [0]
    for i in range(1, len(score_list)):
        score = cummul_score[-1] + score_list[i]
        if score < 0:
            cummul_score.append(0)
        else:
            cummul_score.append(score)
            if not start:
                    # trim_start = value when cummulative score is first > 0
                trim_start = i
                start = True
        # trim_finish = index of highest cummulative score,
        # marking the end of sequence segment with highest cummulative score
    trim_finish = cummul_score.index(max(cummul_score))
    return seq_record[trim_start:trim_finish]

def get_seqobj_from_abi(abi_fp: str) -> [Seq, Seq, list]:
    dna = SeqIO.read(abi_fp, 'abi')
    dna_trimmed = _abi_trim(dna)
    quality_trimmed = dna_trimmed.letter_annotations["phred_quality"]
    return dna, dna_trimmed, quality_trimmed

def get_seq_from_record(dna_obj: Seq, reverse=False) -> [Seq]:
    if not reverse:
        return dna_obj.seq
    else:
        return dna_obj.reverse_complement().seq


def find_match_on_all_h3probes(log: logging, h3_dict: dict, d: SeqIO.SeqRecord, sample:str, vh_abi_dict: dict, vl_abi_dict: dict) -> list:
    """
    returns a list of matched, h3_probe_name and h3_probe_seq, vh,vl abi file path etc. 
    """
    results = []
    for h3key, h3val in h3_dict.items():
        seq_f = get_seq_from_record(d, reverse=False)
        seq_r = get_seq_from_record(d, reverse=True)
        vh_fn = vh_abi_dict.get(sample)
        vl_fn = vl_abi_dict.get(sample)
        if h3val in seq_f:
            log.info(f"[+] INFO: Forward_Strand_Match|{h3key}|{sample}|{vh_fn}|{vl_fn}")
            # print(f"Forward_Strand_Match|{h3key}|{sample}|{vh_fn}|{vl_fn}|{h3val}|{seq_f}")
            results.append(["Forward_Strand_Match", h3key, sample, vh_fn, vl_fn, h3val, seq_f, d])
        elif h3val in seq_r:
            log.info(f"[+] INFO: Reverse_Strand_Match|{h3key}|{sample}|{vh_fn}|{vl_fn}")
            # print(f"Reverse_Strand_Match|{h3key}|{sample}|{vh_fn}|{vl_fn}|{h3val}|{seq_r}")
            results.append(["Reverse_Strand_Match", h3key, sample, vh_fn, vl_fn, h3val, seq_r, d])
        else:
            pass
            # log.info(f"[-] INFO: No_match|{h3key}|{sample}|{vh_fn}|{vl_fn}")
            # results.append(["No_match", h3key, sample, vh_fn, vl_fn, '', ''])
    return(results)


def copy_mtched_abi_files_to_resdir(log: logging, df_vh:pd.DataFrame, df_vl:pd.DataFrame, res_dir_vh:str, res_dir_vl:str)->pd.DataFrame:
    """
    creates a dir within the results_dir for VH and VL, with h3_name
    and copy vh and vl matched abi files into it
    """
    resx = []
    for e,i in enumerate(df_vh.h3_name.unique()[:], start=1):
        _dfvh = df_vh[df_vh.h3_name == i]
        _dfvl = df_vl[df_vl.h3_name == i]
        for j in range(_dfvh.shape[0]):
            __dfvh = _dfvh.iloc[j]
            __dfvl = _dfvl.iloc[j]
            # creating dirs 
            res_dir_vh_msid = checking_dirs(f"{res_dir_vh}/{__dfvh.h3_name}", log, log_msg=False, create_dir=True)
            res_dir_vl_msid = checking_dirs(f"{res_dir_vl}/{__dfvl.h3_name}", log, log_msg=False, create_dir=True)
            shutil.copy(__dfvh.vh_abi_fp, res_dir_vh_msid)
            shutil.copy(__dfvl.vl_abi_fp, res_dir_vl_msid)
            log.info(f"[+] INFO: copying {__dfvh.vh_abi_fp} to {res_dir_vh_msid}")
            log.info(f"[+] INFO: copying {__dfvh.vl_abi_fp} to {res_dir_vl_msid}")
            resx.append([__dfvh.h3_name, __dfvh.sample_id, f"{res_dir_vh}/{__dfvh.sample_id}", __dfvh.vh_abi_fp])
            resx.append([__dfvl.h3_name, __dfvl.sample_id, f"{res_dir_vl}/{__dfvl.sample_id}", __dfvl.vl_abi_fp])
    return pd.DataFrame(resx, columns=["h3_name", "sample_id", "abi_out_loc", "abi_initial_filepath"])


def copy_mtched_abi_files_to_resdir2(res_dir_vx:str, vx_abi_fp:str, log: logging, log_msg:bool= True, df_vh:pd.DataFrame = None, df_vl:pd.DataFrame = None)->pd.DataFrame:
    """
    creates a dir within the results_dir for VH and VL, with h3_name
    and copy vh and vl matched abi files into it
    """
    def _cp_files(log: logging, df:pd.DataFrame, res_dir_vx:str, vx_abi_fp:str):
        res = []
        for i in df.h3_name.unique()[:]:
            _df = df[df.h3_name == i]
            for j in range(_df.shape[0]):
                __df = _df.iloc[j]
                res_dir_msid = checking_dirs(f"{res_dir_vx}/{__df.h3_name}", log, log_msg=False, create_dir=True)
                shutil.copy(__df[vx_abi_fp], res_dir_msid)
                log.info(f"[+] INFO: copying {__df[vx_abi_fp]} to {res_dir_msid}") if log_msg else None
                res.append([__df.h3_name, __df.sample_id, f"{res_dir_vx}/{__df.sample_id}", __df[vx_abi_fp]])
        return pd.DataFrame(res, columns=["h3_name", "sample_id", "abi_out_loc", "abi_initial_filepath"])
    
    df1 = _cp_files(log, df_vh, res_dir_vh, 'vh_abi_fp')
    df2 = _cp_files(log, df_vl, res_dir_vl, 'vl_abi_fp')
    return pd.concat([df1,df2])
        
    
    # resx, resy = [],[]
    # for i df_vh.h3_name.unique()[:]:
    #     _dfvh = df_vh[df_vh.h3_name == i]
    #     for j in range(_dfvh.shape[0]):
    #         __dfvh = _dfvh.iloc[j]
    #         res_dir_vh_msid = checking_dirs(f"{res_dir_vh}/{__dfvh.h3_name}", log, log_msg=False, create_dir=True)
    #         shutil.copy(__dfvh.vh_abi_fp, res_dir_vh_msid)
    #         log.info(f"[+] INFO: copying {__dfvh.vh_abi_fp} to {res_dir_vh_msid}")
    #         resx.append([__dfvh.h3_name, __dfvh.sample_id, f"{res_dir_vh}/{__dfvh.sample_id}", __dfvh.vh_abi_fp])
    # for i in df_vl.h3_name.unique()[:]:
    #     _dfvl = df_vl[df_vl.h3_name == i]
    #     for j in range(_dfvl.shape[0]):
    #         __dfvl = _dfvl.iloc[j]
    #         res_dir_vl_msid = checking_dirs(f"{res_dir_vl}/{__dfvl.h3_name}", log, log_msg=False, create_dir=True)
    #         shutil.copy(__dfvl.vl_abi_fp, res_dir_vl_msid)
    #         log.info(f"[+] INFO: copying {__dfvh.vl_abi_fp} to {res_dir_vl_msid}")
    #         resy.append([__dfvl.h3_name, __dfvl.sample_id, f"{res_dir_vl}/{__dfvl.sample_id}", __dfvl.vl_abi_fp])
    # return pd.DataFrame(resx, columns=["h3_name", "sample_id", "abi_out_loc", "abi_initial_filepath"])

In [9]:
Path('logs').mkdir(parents=True, exist_ok=True)
log = utils.make_logger("logs/rp_log")
log.info("\nSTART")
log.info("\nCHECKING CONFIGURATION FILE CONTENTS...")

In [10]:
config = configparser.ConfigParser()
config.read_file(open('config.ini'))

In [11]:
# checking config file
# if any error occurs then program terminates; then check the config file

abi_sequence_folder = checking_dirs(config['Paths']['abi_sequence_folder'], log, log_msg=True, create_dir=False)
vh_template_sequence_folder = checking_dirs(config['Paths']['vh_template_sequence_folder'], log, log_msg=True, create_dir=False)
vl_template_sequence_folder = checking_dirs(config['Paths']['vl_template_sequence_folder'], log, log_msg=True, create_dir=False)
results_dir = checking_dirs(config['Paths']['results_dir'], log, log_msg=True, create_dir=True)
h3_nt_data_sheet_filepath = check_files(config['Files']['h3_nt_data_sheet_filepath'], log)
df = check_tsv_file(h3_nt_data_sheet_filepath, log)
excel_path_file_name = create_results_excel_file_path(config['Paths']['results_dir'], config['Files']['output_excel_file_name'])

# patters
pat_vh, pat_vl = get_patterns(config)
patrm_vh, patrm_vl = get_patterns_to_rm(config)

# creating dirs for copying matched abi files
res_dir_vh = checking_dirs(f"{results_dir}/{pat_vh}", log, log_msg=True, create_dir=True)
res_dir_vl = checking_dirs(f"{results_dir}/{pat_vl}", log, log_msg=True, create_dir=True)


In [12]:
log.info("\nCHECKING THE AB1 FILES...")

In [13]:
vh_abi_dict = {i.name.replace('.abi', '').replace(patrm_vh, '') : str(i) for i in sorted([*Path(abi_sequence_folder).glob(f"*{pat_vh}*.abi")])}
vl_abi_dict = {i.name.replace('.abi', '').replace(patrm_vl, '') : str(i) for i in sorted([*Path(abi_sequence_folder).glob(f"*{pat_vl}*.abi")])}

In [14]:
sample_ids = checking_ab1_files(vh_abi_dict, vl_abi_dict)

In [15]:
# dictionary containing probe seq and name
h3_dict = df.set_index('name', drop=True).to_dict().get('h3_nt')

In [16]:
# this loop  iternate through all the sample ids and find_match_on_all_h3probes function search for the probe on the normal seq and the rev complemented seq
# if there is a match which will be returened as a list

log.info(f"\nITERATING THROUGH EACH SAMPLE ID")
result_vh, result_vl = [], []
for sample in sample_ids:
    # print(f">>{sample}")
    _vhabi, _vlabi = get_abi_file_path(key=sample, vh_abi_dict=vh_abi_dict, vl_abi_dict=vl_abi_dict)
    vh_d, vh_dtrim, vh_dqtrimlst = get_seqobj_from_abi(_vhabi)
    vl_d, vl_dtrim, vl_dqtrimlst = get_seqobj_from_abi(_vlabi)
    
    # matching each probe on  VH and VL - normal and revcomp sequence
    vh_prob_search = find_match_on_all_h3probes(log, h3_dict, vh_d, sample, vh_abi_dict, vl_abi_dict)
    vl_prob_search = find_match_on_all_h3probes(log, h3_dict, vl_d, sample, vh_abi_dict, vl_abi_dict)
    if len(vh_prob_search) >=1:
        result_vh.append(vh_prob_search)
    if len(vl_prob_search) >=1:
        result_vl.append(vl_prob_search)
log.info(f"\nFINISH ITERATING THROUGH EACH SAMPLE ID")
# if vh and vl ids match then copy the files to a new place
colnames=["Match","h3_name","sample_id","vh_abi_fp","vl_abi_fp","probe_seq", "abi_seq", "seq_record"]
df_vh = pd.DataFrame(chain.from_iterable(result_vh))
df_vl = pd.DataFrame(chain.from_iterable(result_vl))
df_vh.columns, df_vl.columns = colnames, colnames

In [17]:
# if the sahpe of df_vh and df_vl are same and the h3_name in both dataframes are same then copy the matched data to a new dir
if(df_vl.shape == df_vh.shape) and (df_vl.h3_name == df_vh.h3_name).all():
    log.info(f"[+] INFO: There are {df_vl.shape[0]} matches in the dataframe between vh and vl")
    log.info(f"[+] INFO: Of which {df_vh[['vh_abi_fp', 'vl_abi_fp']].drop_duplicates().shape[0]} samples has to be moved to a new dir")
else:
    log.info(f"[+] INFO: There are {df_vh.shape[0]} matches in df_vh")
    log.info(f"[+] INFO: There are {df_vl.shape[0]} matches in df_vl")

In [18]:
df_vl.head()

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,abi_seq,seq_record
0,Forward_Strand_Match,TMH577-hF-014-G03,TMH577-hIgG1-013-A3_A03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCAAGAGGAATTCGCCGTATTACTATGGTTCGGGGAGCTGGGGGGT...,"(A, C, G, G, A, A, G, C, C, G, G, A, A, G, C, ...","(A, C, G, G, A, A, G, C, C, G, G, A, A, G, C, ..."
1,Forward_Strand_Match,TMH577-hF-012-F03,TMH577-hIgG1-013-D10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGAGGGGGCGGATACAGCTATGGCATTGACTAC,"(G, G, G, C, A, T, G, C, C, T, G, G, A, G, G, ...","(G, G, G, C, A, T, G, C, C, T, G, G, A, G, G, ..."
2,Forward_Strand_Match,TMH577-hF-012-C01,TMH577-hIgG1-013-E11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGCGGGTGGGAGCCGCGGCCGATGCTTTTGATATC,"(T, G, G, C, A, G, T, C, C, T, G, A, G, C, A, ...","(T, G, G, C, A, G, T, C, C, T, G, A, G, C, A, ..."
3,Forward_Strand_Match,TMH577-hF-016-C12,TMH577-hIgG1-013-F3_F03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGAGGAGTGACTACGGCCGACTAC,"(G, G, G, C, C, T, G, C, C, T, A, G, C, C, A, ...","(G, G, G, C, C, T, G, C, C, T, A, G, C, C, A, ..."
4,Forward_Strand_Match,TMH577-hF-017-C01,TMH577-hIgG1-013-G3_G03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAATCTGTCTGCGTGGGAAGTACCC,"(G, G, G, G, A, A, A, G, G, C, C, T, G, G, G, ...","(G, G, G, G, A, A, A, G, G, C, C, T, G, G, G, ..."


In [19]:
df_vh.head()

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,abi_seq,seq_record
0,Reverse_Strand_Match,TMH577-hF-012-E03,TMH577-hIgG1-013-A10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGACCCTTTTGGAGCGGCCTAACAAGAGAAAACTACTATTACG...,"(T, C, A, C, G, G, G, G, A, T, T, T, C, C, A, ...","(A, T, G, C, C, A, A, A, G, C, C, C, A, A, G, ..."
1,Reverse_Strand_Match,TMH577-hF-015-C12,TMH577-hIgG1-013-A11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCTAGGTACTACTACGGTATGGACGTC,"(T, T, A, T, G, G, G, A, C, T, T, T, C, C, T, ...","(C, C, G, T, T, C, C, A, A, A, A, C, C, A, A, ..."
2,Reverse_Strand_Match,TMH577-hF-014-B12,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ..."
3,Reverse_Strand_Match,TMH577-hF-015-C08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ..."
4,Reverse_Strand_Match,TMH577-hF-015-D08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ..."


In [20]:
df_vh.seq_record.iloc[10]

SeqRecord(seq=Seq('ACGGTCAAAGCCCCAAGGGTCAAGGCCTGGGCCTGCCCCAGAAAGCTTACTCAC...CAA'), id='40250105', name='TMH577-hIgG1-013-A5_A05_GATC-VH60-2617917.abi', description='', dbxrefs=[])

In [21]:
df_vl.seq_record.iloc[10]

SeqRecord(seq=Seq('GGGCAGGCTGAGGAAGCAGGGTCTGACAGGGAGGGCAGAGGTCCTTGCTCAGGA...CTT'), id='40250607', name='TMH577-hIgG1-014-C10_VL79.abi', description='', dbxrefs=[])

In [27]:
df_vh.shape

(183, 8)

In [28]:
df_vl.shape

(23, 8)

In [26]:

# need to work on this
def copy_mtched_abi_files_to_resdir2(res_dir_vx:str, vx_abi_fp:str, log: logging, log_msg:bool= True, df_vh:pd.DataFrame = None, df_vl:pd.DataFrame = None)->pd.DataFrame:
    """
    creates a dir within the results_dir for VH and VL, with h3_name
    and copy vh and vl matched abi files into it
    """
    def _cp_files(log: logging, df:pd.DataFrame, res_dir_vx:str, vx_abi_fp:str):
        res = []
        for i in df.h3_name.unique()[:]:
            _df = df[df.h3_name == i]
            for j in range(_df.shape[0]):
                __df = _df.iloc[j]
                res_dir_msid = checking_dirs(f"{res_dir_vx}/{__df.h3_name}", log, log_msg=False, create_dir=True)
                shutil.copy(__df[vx_abi_fp], res_dir_msid)
                log.info(f"[+] INFO: copying {__df[vx_abi_fp]} to {res_dir_msid}") if log_msg else None
                res.append([__df.h3_name, __df.sample_id, f"{res_dir_vx}/{__df.sample_id}", __df[vx_abi_fp]])
        return pd.DataFrame(res, columns=["h3_name", "sample_id", "abi_out_loc", "abi_initial_filepath"])
    
    df1 = _cp_files(log, df_vh, res_dir_vh, 'vh_abi_fp')
    df2 = _cp_files(log, df_vl, res_dir_vl, 'vl_abi_fp')
    return pd.concat([df1,df2])

In [25]:
log.info(f"\nCOPY MATCHED ABI FILES INTO NEW LOCATION")
res_df_copy = copy_mtched_abi_files_to_resdir2(log, log_msg=True, df_vh, df_vl, res_dir_vh, res_dir_vl)
log.info(f"\nFINISH COPYING FILES")

AttributeError: 'str' object has no attribute 'h3_name'

In [31]:
res_df_copy.h3_name.value_counts()

TMH577-hF-014-B12    13
TMH577-hF-015-C08    13
TMH577-hF-015-D08    13
TMH577-hF-017-G04    13
TMH577-hF-012-E03     7
                     ..
TMH577-hF-017-H10     1
TMH577-hF-016-D06     1
TMH577-hF-016-C03     1
TMH577-hF-004-G09     1
TMH577-hF-017-D03     1
Name: h3_name, Length: 67, dtype: int64

### Note
- same TMH577-hF-012-E03 for eg: matches TMH577-hIgG1-014-H6_H06, TMH577-hIgG1-014-G4_G04, TMH577-hIgG1-014-A5_A05, TMH577-hIgG1-013-H7_H07, TMH577-hIgG1-013-H3_H03, TMH577-hIgG1-013-A10
- todo
    - create a dict for template_gp_for_vh and vl
    - copy the abi files to a new dir


In [29]:
# for template_strand in os.listdir(vh_template_sequence_folder):
#     parsed_template_strand = template_strand.split('VH-')[1].split('.gb')[0]
#     print(f"{template_strand} | {parsed_template_strand}")